<a href="https://colab.research.google.com/github/MehrdadJalali-AI/GAN-MOF/blob/main/GraphGANSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import collections
import tqdm
import multiprocessing
import numpy as np
import tensorflow.compat.v1 as tf
import pickle

# Disable eager execution for compatibility with older TensorFlow
tf.disable_eager_execution()

# GraphGAN Configuration
class Config:
    n_node = 100  # Number of nodes in the graph
    n_emb = 50  # Embedding dimension
    batch_size_gen = 64
    batch_size_dis = 64
    n_sample_gen = 20
    lr_gen = 1e-3
    lr_dis = 1e-3
    n_epochs = 5  # Reduce epochs for faster testing
    n_epochs_gen = 5
    n_epochs_dis = 5
    update_ratio = 1
    multi_processing = False
    window_size = 2
    save_steps = 2
    load_model = False

config = Config()

# Sample Graph Data
class SampleGraph:
    def __init__(self, n_nodes=100, edge_prob=0.1):
        self.n_nodes = n_nodes
        self.graph = self.generate_random_graph()

    def generate_random_graph(self):
        graph = {i: [] for i in range(self.n_nodes)}
        for i in range(self.n_nodes):
            for j in range(i + 1, self.n_nodes):
                if np.random.rand() < 0.1:
                    graph[i].append(j)
                    graph[j].append(i)
        return graph

sample_graph = SampleGraph(n_nodes=config.n_node).graph

# Generator Model
class Generator:
    def __init__(self, config):
        self.n_node = config.n_node
        self.n_emb = config.n_emb
        self.embedding_matrix = tf.get_variable("embedding", shape=[self.n_node, self.n_emb], initializer=tf.random_normal_initializer())
        self.node_id = tf.placeholder(tf.int32, shape=[None])
        self.node_neighbor_id = tf.placeholder(tf.int32, shape=[None])
        self.reward = tf.placeholder(tf.float32, shape=[None])
        self.node_embedding = tf.nn.embedding_lookup(self.embedding_matrix, self.node_id)
        self.node_neighbor_embedding = tf.nn.embedding_lookup(self.embedding_matrix, self.node_neighbor_id)
        self.score = tf.reduce_sum(tf.multiply(self.node_embedding, self.node_neighbor_embedding), axis=1)
        self.loss = -tf.reduce_mean(tf.log_sigmoid(self.score) * self.reward)
        self.g_updates = tf.train.AdamOptimizer(config.lr_gen).minimize(self.loss)

# Discriminator Model
class Discriminator:
    def __init__(self, config):
        self.n_node = config.n_node
        self.n_emb = config.n_emb
        self.embedding_matrix = tf.get_variable("embedding_d", shape=[self.n_node, self.n_emb], initializer=tf.random_normal_initializer())
        self.node_id = tf.placeholder(tf.int32, shape=[None])
        self.node_neighbor_id = tf.placeholder(tf.int32, shape=[None])
        self.label = tf.placeholder(tf.float32, shape=[None])
        self.node_embedding = tf.nn.embedding_lookup(self.embedding_matrix, self.node_id)
        self.node_neighbor_embedding = tf.nn.embedding_lookup(self.embedding_matrix, self.node_neighbor_id)
        self.score = tf.reduce_sum(tf.multiply(self.node_embedding, self.node_neighbor_embedding), axis=1)
        self.loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.label, logits=self.score))
        self.d_updates = tf.train.AdamOptimizer(config.lr_dis).minimize(self.loss)

# GraphGAN Training
class GraphGAN:
    def __init__(self, config):
        self.config = config
        self.generator = Generator(config)
        self.discriminator = Discriminator(config)
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def train(self):
        print("Starting training...")
        for epoch in range(self.config.n_epochs):
            print(f"Epoch {epoch+1}/{self.config.n_epochs}")
            for _ in range(self.config.n_epochs_dis):
                node_pairs, labels = self.get_discriminator_data()
                self.sess.run(self.discriminator.d_updates, feed_dict={
                    self.discriminator.node_id: [pair[0] for pair in node_pairs],
                    self.discriminator.node_neighbor_id: [pair[1] for pair in node_pairs],
                    self.discriminator.label: labels
                })
            for _ in range(self.config.n_epochs_gen):
                node_pairs, rewards = self.get_generator_data()
                self.sess.run(self.generator.g_updates, feed_dict={
                    self.generator.node_id: [pair[0] for pair in node_pairs],
                    self.generator.node_neighbor_id: [pair[1] for pair in node_pairs],
                    self.generator.reward: rewards
                })
        print("Training complete.")

    def get_discriminator_data(self):
        node_pairs = []
        labels = []
        for node in range(self.config.n_node):
            for neighbor in sample_graph[node]:
                node_pairs.append((node, neighbor))
                labels.append(1)
                negative_sample = np.random.randint(0, self.config.n_node)
                while negative_sample in sample_graph[node]:
                    negative_sample = np.random.randint(0, self.config.n_node)
                node_pairs.append((node, negative_sample))
                labels.append(0)
        return node_pairs, labels

    def get_generator_data(self):
        node_pairs = []
        rewards = []
        for node in range(self.config.n_node):
            sampled_neighbors = np.random.choice(self.config.n_node, self.config.n_sample_gen, replace=True)
            for neighbor in sampled_neighbors:
                node_pairs.append((node, neighbor))
                rewards.append(np.random.rand())
        return node_pairs, rewards

# Run GraphGAN
if __name__ == "__main__":
    graphgan = GraphGAN(config)
    graphgan.train()


Starting training...
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
Training complete.
